In [0]:
!python --version

Python 3.6.9


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

seed = 0

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
data = pd.read_csv('train_preprocess.csv', sep=';')
data.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,Year,Month,Day,WindSpeedDelta,WindDirChange,HumidityDelta,PressureDelta,CloudDelta,TempDelta
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,False,2008,12,1,4.0,True,-49.0,-0.6,NaN,4.9
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,False,2008,12,2,18.0,True,-19.0,-2.8,NaN,7.1
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,False,2008,12,3,7.0,True,-8.0,1.1,NaN,2.2
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,False,2008,12,4,-2.0,True,-29.0,-4.8,NaN,8.4
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,False,2008,12,5,13.0,True,-49.0,-4.8,1.0,11.9


In [0]:
y = data.RainTomorrow
data.drop('RainTomorrow', axis=1, inplace=True)

In [10]:
data = data[['MinTemp', 'MaxTemp', 'Rainfall', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp3pm', 'RainTomorrow']]
data.head()
data.shape

(70240, 9)

In [11]:
data = data.dropna(how='any')
data.shape

(61162, 9)

In [0]:
categorical_cols = [cat for cat in data.columns if data[cat].dtype == 'O']
numerical_cols = list(set(data.columns.values.tolist()) - set(categorical_cols))

In [0]:
numeric_means = data[numerical_cols].mean()
X_real_mean = data[numerical_cols].fillna(numeric_means)
X_cat = data[categorical_cols].fillna('NaN')

In [0]:
from sklearn.feature_extraction import DictVectorizer as DV

encoder = DV(sparse = False)
X_cat_oh = encoder.fit_transform(X_cat.astype('str').T.to_dict().values())

In [0]:
X_cat_oh.shape

(70240, 81)

In [0]:
X = np.hstack((X_real_mean, X_cat_oh))
X.shape

(70240, 106)

In [0]:
from sklearn.model_selection import train_test_split

(X_train, 
 X_test, 
 y_train, y_test) = train_test_split(data, y, 
                                     test_size=0.2, 
                                     random_state=seed,
                                    stratify=y)

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
y_train = y_train.values
y_test = y_test.values

In [16]:
y_test

array([False, False, False, ..., False, False, False])

In [17]:
import keras
from keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU
from keras.models import Sequential
from keras import backend as K
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [46]:
model = Sequential()
model.add(Dense(128, init='he_normal', input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Dense(64, init='he_normal'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Dense(132, init='he_normal'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1])
checkpoint = ModelCheckpoint("./nn_*2dense_dr0.2_ep100_he_init-{epoch:02d}-{val_f1:.2f}.hdf5",
                             monitor='val_f1', save_best_only=True, mode='max', period=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(8,), kernel_initializer="he_normal")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_initializer="he_normal")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(132, kernel_initializer="he_normal")`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 64)                6848      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 32)               

In [47]:
model_history = model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.1, callbacks=[checkpoint], shuffle=True)

Train on 44036 samples, validate on 4893 samples
Epoch 1/100
44036/44036 [==============================] - 2s 50us/step - loss: 0.4292 - accuracy: 0.8145 - f1: 0.5148 - val_loss: 0.3697 - val_accuracy: 0.8398 - val_f1: 0.5949
Epoch 2/100
44036/44036 [==============================] - 2s 35us/step - loss: 0.4019 - accuracy: 0.8293 - f1: 0.5385 - val_loss: 0.3674 - val_accuracy: 0.8475 - val_f1: 0.5854
Epoch 3/100
44036/44036 [==============================] - 2s 34us/step - loss: 0.3939 - accuracy: 0.8318 - f1: 0.5462 - val_loss: 0.3626 - val_accuracy: 0.8459 - val_f1: 0.5934
Epoch 4/100
44036/44036 [==============================] - 2s 35us/step - loss: 0.3896 - accuracy: 0.8333 - f1: 0.5529 - val_loss: 0.3607 - val_accuracy: 0.8490 - val_f1: 0.5908
Epoch 5/100
44036/44036 [==============================] - 2s 35us/step - loss: 0.3860 - accuracy: 0.8348 - f1: 0.5566 - val_loss: 0.3599 - val_accuracy: 0.8457 - val_f1: 0.5916
Epoch 6/100
44036/44036 [==============================] - 2s

In [26]:
model.predict(X_test)[:, 0]

array([0.04584482, 0.10324436, 0.07779029, ..., 0.05173403, 0.27866855,
       0.11530078], dtype=float32)

In [48]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, model.predict(X_test))

0.8559712478035064

In [49]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_test, model.predict(X_test) >=0.5)

array([[8995,  466],
       [1464, 1308]])

In [0]:
m = keras.models.load_model('nn-39-0.63.hdf5', custom_objects={'f1': f1})
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1])

In [0]:
from keras import models
m = models.load_model('/content/nn-39-0.63.hdf5', custom_objects={'f1': f1})
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1])

In [0]:
roc_auc_score(y_test, m.predict(X_test))

0.88729887272617

In [0]:
confusion_matrix(y_test, m.predict(X_test) >=0.5)

array([[10280,   515],
       [ 1512,  1741]])

In [0]:
learn_rate = [0.0001, 0.001, 0.01]
epochs = [10, 30, 50]

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(learning_rate):
    model = Sequential()
    model.add(Dense(128, init='he_normal', input_shape=(106,)))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dropout(0.2))
    model.add(Dense(64, init='he_normal'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dropout(0.2))
    model.add(Dense(16, init='he_normal'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    optim = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optim, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [0]:
model = KerasClassifier(build_fn=create_model, batch_size=256, epochs=100)
param_grid = dict(epochs=epochs, learning_rate=learn_rate)
search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
search_result  = search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(106,), kernel_initializer="he_normal")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_initializer="he_normal")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, kernel_initializer="he_normal")`
  


Epoch 1/10
37461/37461 [==============================] - 1s 37us/step - loss: 0.7012 - accuracy: 0.6027
Epoch 2/10
37461/37461 [==============================] - 1s 23us/step - loss: 0.6299 - accuracy: 0.6612
Epoch 3/10
37461/37461 [==============================] - 1s 24us/step - loss: 0.5810 - accuracy: 0.7063
Epoch 4/10
37461/37461 [==============================] - 1s 24us/step - loss: 0.5445 - accuracy: 0.7367
Epoch 5/10
37461/37461 [==============================] - 1s 24us/step - loss: 0.5238 - accuracy: 0.7497
Epoch 6/10
37461/37461 [==============================] - 1s 24us/step - loss: 0.5042 - accuracy: 0.7677
Epoch 7/10
37461/37461 [==============================] - 1s 23us/step - loss: 0.4855 - accuracy: 0.7788
Epoch 8/10
37461/37461 [==============================] - 1s 25us/step - loss: 0.4729 - accuracy: 0.7887
Epoch 9/10
37461/37461 [==============================] - 1s 23us/step - loss: 0.4613 - accuracy: 0.7943
Epoch 10/10
18731/18731 [==============================